# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.122318e+02     1.565452e+00
 * time: 0.09536409378051758
     1     1.086646e+01     1.061690e+00
 * time: 1.0137619972229004
     2    -1.178703e+01     1.100925e+00
 * time: 1.1424801349639893
     3    -3.385328e+01     7.721938e-01
 * time: 1.4513180255889893
     4    -4.724548e+01     5.962187e-01
 * time: 1.910114049911499
     5    -5.682023e+01     1.908494e-01
 * time: 2.058990001678467
     6    -5.972318e+01     1.372165e-01
 * time: 2.1799490451812744
     7    -6.085849e+01     5.156289e-02
 * time: 2.3018441200256348
     8    -6.131510e+01     5.118233e-02
 * time: 2.422210931777954
     9    -6.156307e+01     4.147966e-02
 * time: 2.53706693649292
    10    -6.178187e+01     2.972093e-02
 * time: 2.6586639881134033
    11    -6.194825e+01     2.298466e-02
 * time: 2.7790701389312744
    12    -6.203477e+01     1.730676e-02
 * time: 2.900775909423828
    13    -6.209594e+01     1.371984e-02
 * time: 3.015986919403076

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671063
    AtomicLocal         -18.8557684
    AtomicNonlocal      14.8522658
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666461562
